In [1]:
import pandas as pd
import numpy as np

from parse_output import parse_output

In [2]:
path = 'output/results2.csv'
df = parse_output(path)

In [3]:
def get_ip_asn_frequencies(df):
    def frequency_dict(s):
        v_counts = s.value_counts().to_dict()
        if v_counts == {}:
            v_counts = np.nan
        return v_counts
    
    df = df.copy().set_index('dest name')
    return df.filter(regex='(ip)|(asn)', axis=1).groupby('dest name').agg(frequency_dict)

def drop_empty(df):
    ip_col = 'hop {} probe 1 ip'
    asn_col = 'hop {} probe 1 asn'

    to_drop = []
    for i in range(10, 31):
        if df[ip_col.format(i)].isnull().all():
            to_drop.append(ip_col.format(i))
            to_drop.append(asn_col.format(i))
        
    return df.drop(columns=to_drop)

In [ ]:
target_df = drop_empty(get_ip_asn_frequencies(df))

In [ ]:
import time
df['time'] = pd.to_datetime(df['time'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(x))))

In [ ]:
import plotly.express as px

def sort_ips(col):
    def parse_ip(x):
        if x is not None:
            return list(map(int, x.split('.')))
        else:
            return [0]
        
    ordered = sorted(col, key=lambda x: parse_ip(x))
    ordered.reverse()
    return ordered


def ips_with_time(df):
    cols = ['dest ip']
    cols += ['hop {} probe 1 ip'.format(i) for i in range(1, 31)]
    for col in cols:
        graph_df = df[['dest name', 'time', col]]
        if graph_df[col].isnull().all():
            continue
        fig = px.scatter(graph_df, x='time', y=col, color='dest name', symbol='dest name', category_orders={col: sort_ips(graph_df[col].unique())})
        fig.show()


def target_dest_ips_with_time(df, target_dest):
    cols = ['dest ip']
    cols += ['hop {} probe 1 ip'.format(i) for i in range(1, 31)]
    for col in cols:
        graph_df = df.loc[df['dest name'] == target_dest][['time', col]]
        if graph_df[col].isnull().all():
            continue
        fig = px.scatter(graph_df, x='time', y=col, category_orders={col: sort_ips(graph_df[col].unique())})
        fig.show()

In [ ]:
ips_with_time(df)

In [ ]:
target_dest_ips_with_time(df, 'youtube.com')